# 🔄 Resume Training: English-Vietnamese Model

Resume training from existing checkpoint.

**Prerequisites:**
- Checkpoint file (`best_model.pt` hoặc `checkpoint_epoch_X.pt`)
- Tokenizer files (`tokenizer_en.model`, `tokenizer_vi.model`)
- Processed data (`train.pt`, `val.pt`)

---

## 1. ⚙️ Setup

In [ ]:
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard pyyaml

In [ ]:
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
    f.write(f'HF_TOKEN={HF_TOKEN}\n')
print('✓ .env created')

## 2. 🔧 Configure Paths

**Nhập trực tiếp đường dẫn đến các files trên Kaggle:**

In [ ]:
# ============================================================
# 📁 NHẬP ĐƯỜNG DẪN TRỰC TIẾP ĐẾN CÁC FILES TRÊN KAGGLE
# ============================================================

# Đường dẫn đến checkpoint để resume (có thể là best_model.pt hoặc checkpoint_epoch_X.pt)
CHECKPOINT_PATH = "/kaggle/input/YOUR_DATASET/best_model.pt"  # THAY ĐỔI

# Đường dẫn đến tokenizers
TOKENIZER_SRC_PATH = "/kaggle/input/YOUR_DATASET/tokenizer_en.model"  # THAY ĐỔI
TOKENIZER_TGT_PATH = "/kaggle/input/YOUR_DATASET/tokenizer_vi.model"  # THAY ĐỔI

# Đường dẫn đến processed data
TRAIN_DATA_PATH = "/kaggle/input/YOUR_DATASET/train.pt"  # THAY ĐỔI
VAL_DATA_PATH = "/kaggle/input/YOUR_DATASET/val.pt"  # THAY ĐỔI

# Output directory (nơi lưu checkpoints mới)
OUTPUT_CHECKPOINT_DIR = "/kaggle/working/checkpoints"
OUTPUT_LOG_DIR = "/kaggle/working/logs"

# ============================================================
print("📁 Input Paths:")
print(f"  📦 Checkpoint: {CHECKPOINT_PATH}")
print(f"  📝 Tokenizer EN: {TOKENIZER_SRC_PATH}")
print(f"  📝 Tokenizer VI: {TOKENIZER_TGT_PATH}")
print(f"  📊 Train data: {TRAIN_DATA_PATH}")
print(f"  📊 Val data: {VAL_DATA_PATH}")
print(f"\n📁 Output Paths:")
print(f"  💾 Checkpoints: {OUTPUT_CHECKPOINT_DIR}")
print(f"  📈 Logs: {OUTPUT_LOG_DIR}")

In [ ]:
# Verify input files exist
import os

files_to_check = [
    ("Checkpoint", CHECKPOINT_PATH),
    ("Tokenizer EN", TOKENIZER_SRC_PATH),
    ("Tokenizer VI", TOKENIZER_TGT_PATH),
    ("Train data", TRAIN_DATA_PATH),
    ("Val data", VAL_DATA_PATH),
]

all_exist = True
for name, path in files_to_check:
    exists = os.path.exists(path)
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")
    if not exists:
        all_exist = False

if all_exist:
    print("\n✅ All input files found!")
else:
    print("\n❌ Some files not found. Please check the paths above.")

## 3. 📂 Setup Directories & Symlinks

In [ ]:
import os
import shutil

# Create output directories
os.makedirs(OUTPUT_CHECKPOINT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_CHECKPOINT_DIR}/tokenizers", exist_ok=True)
os.makedirs(OUTPUT_LOG_DIR, exist_ok=True)
os.makedirs("/kaggle/working/data/processed", exist_ok=True)

# Copy tokenizers to output dir (needed for saving)
shutil.copy(TOKENIZER_SRC_PATH, f"{OUTPUT_CHECKPOINT_DIR}/tokenizers/tokenizer_en.model")
shutil.copy(TOKENIZER_TGT_PATH, f"{OUTPUT_CHECKPOINT_DIR}/tokenizers/tokenizer_vi.model")

# Create symlinks for data (avoid copying large files)
if not os.path.exists("/kaggle/working/data/processed/train.pt"):
    os.symlink(TRAIN_DATA_PATH, "/kaggle/working/data/processed/train.pt")
if not os.path.exists("/kaggle/working/data/processed/val.pt"):
    os.symlink(VAL_DATA_PATH, "/kaggle/working/data/processed/val.pt")

print("✓ Directories created")
print("✓ Tokenizers copied")
print("✓ Data symlinks created")

In [ ]:
# Update config with paths
import yaml

CONFIG_FILE = 'config/config.yaml'

with open(CONFIG_FILE, 'r') as f:
    cfg = yaml.safe_load(f)

cfg['paths'] = {
    'data_dir': '/kaggle/working/data',
    'checkpoint_dir': OUTPUT_CHECKPOINT_DIR,
    'log_dir': OUTPUT_LOG_DIR
}
cfg['data']['source'] = 'processed'

with open(CONFIG_FILE, 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False, allow_unicode=True)

print('✓ Config updated')

## 4. 🔍 Verify Checkpoint

In [ ]:
import sys
sys.path.insert(0, '.')

import torch

# Load and inspect checkpoint
checkpoint = torch.load(CHECKPOINT_PATH, map_location='cpu')

print("📦 Checkpoint Info:")
print(f"  Epoch: {checkpoint.get('epoch', 'N/A')}")
print(f"  Val Loss: {checkpoint.get('val_loss', 'N/A'):.4f}" if 'val_loss' in checkpoint else "  Val Loss: N/A")
print(f"  Best Val Loss: {checkpoint.get('best_val_loss', 'N/A'):.4f}" if 'best_val_loss' in checkpoint else "  Best Val Loss: N/A")
print(f"  Has scheduler state: {'scheduler_state_dict' in checkpoint}")
print(f"  Keys: {list(checkpoint.keys())}")

## 5. 🏋️ Resume Training

In [ ]:
from src.train import Trainer
from src.config import load_config

config = load_config('config/config.yaml')
trainer = Trainer(config_path='config/config.yaml')
trainer.setup()

print("\n" + "="*50)
print("🔄 Resuming EN → VI Training")
print("="*50)

# Resume từ checkpoint đã chỉ định
trainer.train(resume_from=CHECKPOINT_PATH)